In [1]:
from random import randint, choice
import tweepy, time, webbrowser, re

class MarkovBot:
    def __init__(self): # path to corpus
        self.sleep_timer = int(60 * 60 * 4)
        self.corpus = {}
        self.start_words = []
        self.end_words = []
        self.api = tweepy.API()
        
    def oauth(self, path_to_secret):
        with open(path_to_secret) as f:
            api_key_secret = f.readlines()
        api_key_secret = [x.strip() for x in api_key_secret] 
        CONSUMER_KEY = api_key_secret[0]
        CONSUMER_SECRET = api_key_secret[1]
        auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth_url = auth.get_authorization_url()
        webbrowser.open(auth_url)
        verifier = raw_input('PIN: ').strip()
        auth.get_access_token(verifier)
        with open('./access_key_secret.txt', 'w') as f:
            f.write(auth.access_token + '\n' + auth.access_token_secret)
        self.api = tweepy.API(auth)
        
    def init_corpus(self, path_to_corpus):
        with open(path_to_corpus, 'r') as f:
            for line in f.readlines():
                english = r'[A-Z|a-z]+$'
                bad_chars = r'(\w)*(\{[^}]+)|(\$)+|(\\)+|(\/)+(\w)*'
                garbage = r'{.jpg}+|{.png}+'
                if re.search(english, line) or re.search(bad_chars, line) or re.search(garbage, line):
                    continue
                words = line.split()
                self.start_words.append(words[0])
                self.end_words.append(words[len(words) - 1])
                n = len(words)
                for word1, word2 in zip(words[:(n - 1)], words[1:]):
                    if word1 in self.corpus:
                        self.corpus[word1].append(word2)
                    else:
                        self.corpus[word1] = [word2]
                        
    def generate(self):
        current_word = choice(self.start_words)
        result = [current_word]
        while current_word in self.corpus.keys() and self.corpus[current_word]:
            current_word = choice(self.corpus[current_word])
            result.append(current_word)
        return ' '.join(result)
    
    def get_tweet(self):
        tweet = self.generate()
        while len(tweet) > 140:
            tweet = self.generate()
        return tweet
        
    def run(self):
        self.init_corpus('./text.txt')
        while True:
            tweet = self.get_tweet() # Samples
            print tweet
            self.api.update_status(tweet) # Posts to twitter
            time.sleep(self.sleep_timer)

In [2]:
bot = MarkovBot()
bot.oauth('./api_key_secret.txt')
bot.init_corpus('./text.txt')

PIN: 3569851


In [ ]:
bot.run()

О корректности задачи упорядочения организации вагонопотоков
